In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from gensim.models import KeyedVectors

In [3]:
w2v=KeyedVectors.load("/content/gdrive/MyDrive/FYP/IMDb/Word2Vec/w2c")

In [4]:
import pandas as pd
import numpy as np
url="/content/gdrive/MyDrive/FYP/IMDb/IMDB-preprocessed.csv"
df=pd.read_csv(url)
y=df["sentiment"]


In [5]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

2511


In [6]:
import tensorflow as tf
maxlen=66
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])

In [7]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in w2v:
    embed_vector=w2v[word]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [8]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.17750642, -1.21290028, -0.95585269, ...,  0.432291  ,
        -0.217326  , -0.69201577],
       [ 1.5916748 ,  0.41039303,  0.79539949, ...,  1.64110196,
         0.89807343,  0.49340928],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,LSTM,Dense
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=66,embeddings_initializer=Constant(embed_matrix)))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False)) 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(1,activation='sigmoid'))

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=64, verbose=1)

Epoch 1/5
586/586 [==============================] - 49s 67ms/step - loss: 0.5017 - accuracy: 0.7604
Epoch 2/5
586/586 [==============================] - 39s 67ms/step - loss: 0.3581 - accuracy: 0.8650
Epoch 3/5
586/586 [==============================] - 39s 67ms/step - loss: 0.2605 - accuracy: 0.9063
Epoch 4/5
586/586 [==============================] - 39s 67ms/step - loss: 0.1739 - accuracy: 0.9454
Epoch 5/5
586/586 [==============================] - 39s 66ms/step - loss: 0.1007 - accuracy: 0.9697


In [14]:
y_pred=model.predict(X_test)
for i in range(len(y_pred)):
  if y_pred[i]>0.5:
    y_pred[i]=1
  else:
    y_pred[i]=0

In [15]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
display_metrics(y_test,y_pred)

0.85512
0.8549677509462332
0.8556348394960872
0.8548271590425232
